## New CDH data from Anna

In [3]:
import os
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

In [4]:
cdh = pd.read_excel(cdh_path)

## Label ICU stays

In [5]:
cdh['Patient_id_2'] = [str(r.ir_id)+'/'+str(r.ICU_rank) for i,r in cdh.iterrows()]
cdh['patient_day'] = [str(r.ir_id)+'/'+str(r.ICU_rank)+'/'+str(r.ICU_day) for i,r in cdh.iterrows()]

In [6]:
cdh = cdh.sort_values(by=['ir_id','ICU_rank','ICU_day'])

In [7]:
errant_stays = []
for p in cdh.Patient_id_2.unique():
    sub = cdh[cdh.Patient_id_2 == p]
    days = sub.day_bucket_ends.values
    i = len(sub) - 1
    while i > 0:
        delta = days[i] - days[i-1]
        num_delta = delta.astype('timedelta64[D]').astype(int)
        if num_delta != 1 and p not in errant_stays:
            errant_stays.append(p)
        i -= 1

In [8]:
len(errant_stays)

7

I went through and looked at all of these. There seems to have been some errors in the ICU_rank and ICU_day numbers. I'm going to sort these stays by date, and every time there's a jump in days I'm going to relabel the ICU rank and start ICU day over. Will take all the patients data so that I'm correctly numbering ICU_rank

In [9]:
for errant_stay in errant_stays:
    p = cdh[cdh.Patient_id_2 == errant_stay].ir_id.values[0]
    sub = cdh[cdh.ir_id == p].sort_values(by='day_bucket_starts')
    rank = sub.ICU_rank.values[0]
    day = 1
    ranks = [rank]
    icu_days = [1]
    days = sub.day_bucket_ends.values
    i = 1
    while i < len(days):
        delta = days[i] - days[i-1]
        num_delta = delta.astype('timedelta64[D]').astype(int)
        if num_delta != 1:
            rank += 1
            day = 1
        else:
            day += 1
        icu_days.append(day)
        ranks.append(rank)
        i += 1
    cdh.loc[sub.index,'ICU_rank'] = ranks
    cdh.loc[sub.index,'ICU_day'] = icu_days

In [10]:
cdh['Patient_id_2'] = [str(r.ir_id)+'/'+str(r.ICU_rank) for i,r in cdh.iterrows()]
cdh['patient_day'] = [str(r.ir_id)+'/'+str(r.ICU_rank)+'/'+str(r.ICU_day) for i,r in cdh.iterrows()]

In [11]:
cdh = cdh.sort_values(by=['ir_id','ICU_rank','ICU_day'])

Rerun code that checks for data discrepencies in dataset

In [12]:
errant_stays = []
for p in cdh.Patient_id_2.unique():
    sub = cdh[cdh.Patient_id_2 == p]
    days = sub.day_bucket_ends.values
    i = len(sub) - 1
    while i > 0:
        delta = days[i] - days[i-1]
        num_delta = delta.astype('timedelta64[D]').astype(int)
        if num_delta != 1 and p not in errant_stays:
            errant_stays.append(p)
        i -= 1

In [13]:
len(errant_stays)

0

Looks good

Will call our dataset cd (even though this contains some days outside of CarpeDiem)

In [21]:
cd = pd.read_csv(cd_path,index_col=0)

## Clean up column names

In [22]:
# Marjorie's list to fix column names, will want to reverse this

rename_list = {#'script_id':'pt_study_id',
               'stay':'ICU_rank',
                'day':'ICU_day',
               'SOFA_score':'SOFA',
               'Intubation_flag':'intub_flag',
               'Hemodialysis_flag':'hd_flag',
               'Temperature':'Temperature_avg',
               'Heart_rate':'Heart_rate_avg',
               'Systolic_blood_pressure':'Systolic_blood_pressure_avg',
               'Diastolic_blood_pressure':'Diastolic_blood_pressure_avg',
               'Mean_arterial_pressure':'Mean_arterial_pressure_avg',## double check
               'Norepinephrine_rate':'norepinephrine_max',
               'Respiratory_rate':'Respiratory_rate_avg',
               'Oxygen_saturation':'Oxygen_saturation_avg',
               'GCS_eye_opening':'Eye_Opening_min',
               'GCS_motor_response':'Best_Motor_Response_min',
               'GCS_verbal_response':'Best_Verbal_Response_min',
                'Minute_Ventilation':'Minute_Ventilation_avg',
#                'RASS_score':'RASS_avg',
               'PEEP':'Peep_avg',
               'FiO2':'FiO2_avg',
               'Plateau_Pressure':'Plateau_Pressure_avg',
    
               'PaO2FIO2_ratio':'pO2_FiO2_ratio_min',
               'WBC_count':'wbc_avg',
               'Lymphocytes':'abs_lymphocytes_avg',
               'Neutrophils':'abs_neutrophils_avg',
               'Hemoglobin':'hemoglobin_avg',
               'Platelets':'PLATELET_min',
               'Bicarbonate':'BICARBONATE_avg',
               'Creatinine':'Creatinine_max',
               'Albumin':'ALBUMIN_avg',
               'Bilirubin':'BILIRUBIN_max',
               'CRP':'CRP_avg',
               'D_dimer':'d_dimer_avg',
               'Ferritin':'FERRITIN_avg',
               'LDH':'LDH_avg',
               'Lactic_acid':'Lactic_Acid_avg',
               'Procalcitonin':'PROCALCITONIN_avg'}
feature_map = {value: key for key, value in rename_list.items()}


In [23]:
cdh_clean = cdh.copy()

In [24]:
cdh_clean.columns = [feature_map[col] if col in feature_map.keys() else col for col in cdh_clean.columns]

In [25]:
for col in cd.columns:
    if col not in cdh_clean.columns:
        print(col)

pt_study_id
Hemodialysis_flag
RASS_score
Lung_Compliance
ABG_PaO2
ABG_PaCO2
ABG_pH
Norepinephrine_flag


RASS_score, Lung_Compliance, ABG_pH, ABG_PaCO2, ABG_PaO2, and Hemodialysis_flag need to be added

In [26]:
extra_feature_map = {
    'RASS_max':'RASS_score',
    'Static_Compliance_avg':'Lung_Compliance',
    'PH_ART_avg':'ABG_pH',
    'PCO2_ART_avg':'ABG_PaCO2',
    'PO2_ART_avg':'ABG_PaO2',
    'HD_flag':'Hemodialysis_flag'
}

cdh_clean.columns = [extra_feature_map[col] if col in extra_feature_map.keys() else col for col in cdh_clean.columns]


In [27]:
remove_patterns = ["_min","_max",'_avg']
drop_cols = [col for col in cdh_clean.columns if any(col.endswith(combo) for combo in remove_patterns)]
cdh_clean = cdh_clean.drop(columns=drop_cols)

In [28]:
remove_patterns = ["_min","_max",'_avg']
drop_cols = [col for col in cdh_clean.columns if any(col.endswith(combo) for combo in remove_patterns)]
cdh_clean = cdh_clean.drop(columns=drop_cols)

Proceed without outlier adjustment for now.

### Fix norepinephrine

In [30]:
cdh_clean['Norepinephrine_rate'] = [0 if pd.isna(x) else x for x in cdh_clean.Norepinephrine_rate]

In [31]:
cdh_clean.Norepinephrine_rate.isnull().value_counts()

False    5814
Name: Norepinephrine_rate, dtype: int64

Save intermediate file

In [35]:
# cdh_clean.to_csv()